<a href="https://www.kaggle.com/code/jiprud/cnc-data-analysis?scriptVersionId=92182820" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This notebook is my first attempt to analyze some real world dataset.
The aim is to use some visualization tools in order to understand the data in the dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set(rc = {'figure.figsize':(30,16)})
import os

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
# read all experiment files into one dataframe
experiments = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.startswith('experiment'):
            df = pd.read_csv(f"{dirname}/{filename}", index_col=None, header=0)
            df['Experiment'] = int(filename[-6:-4])
            experiments.append(df)
            
series = pd.concat(experiments, axis=0, ignore_index=True)
series    

In [ ]:
# lets see how many machining process steps we have
display(series['Machining_Process'].unique())

I assume that only following steps are the ones where the tool cuts material:
* Layer 1 Up
* Layer 1 Down
* Layer 2 Up
* Layer 2 Down

Other steps, according to my assumption, serve as a preparation steps - move the tool to the right position for machining.

In [ ]:
# add info if a step (point) is a milling one or a service one (preparation, repositioining etc.)
series['Milling'] = series['Machining_Process'].str.startswith('Layer')

# show tool trajectory - only for milling steps
series[series.Milling].plot(x='X1_ActualPosition',y='Y1_ActualPosition');


Figure above shows that my assumption about milling/preparation steps was wrong!
The "Layer..." steps clearly contain also some moving of the tool without the real contact with material.

I need to dig deeper into data to gain better understanding.

For now, lets continue only with experiment 1 as it is the one which meets my expectation...

In [ ]:
# drop all experiments but No.1 (the reason is explained above)
series = series[series.Experiment == 1]

In [ ]:
# add column with actual speed - regardless of direction
series['Velocity'] = np.sqrt(series['X1_ActualVelocity'] ** 2 + series['Y1_ActualVelocity'] ** 2+ series['Z1_ActualVelocity'] ** 2)
series['Velocity'].plot() # all steps - including preparation etc.
series[series.Milling]['Velocity'].plot(); # only milling steps

In [ ]:
# show tool trajectory and use color encoding to display speed of the tool motion

sns.scatterplot(data=series[series.Milling], x='X1_ActualPosition', y='Y1_ActualPosition', hue='Velocity', size='Velocity', sizes=(100, 300));

Lets see if we can find some correlation between columns

In [ ]:
sns.heatmap(series.corr());

## Spindle output power vs Machining process
**Hypothesis**: Output power of the tool (S1_OutputPower) is bigger for "Milling" points than in "Preparation" ones.
Let's see if we can prove this.

In [ ]:
milling_pwr = series[series.Milling]['S1_OutputPower'].mean()
not_milling_pwr = series[series.Milling == False]['S1_OutputPower'].mean()
print('Milling pwr average:' + str(milling_pwr))
print('Not Milling pwr average:' + str(not_milling_pwr))
pct = milling_pwr / not_milling_pwr * 100
pct

**Conclusion**: hypothesis was proven correct.
Average spindle power for Milling steps is two times bigger than for non Milling steps

In [ ]:
# lets see how the spindle power for each process step looks in graph
sns.scatterplot(data=series, x='Machining_Process', y='S1_OutputPower');